<a href="https://colab.research.google.com/github/mateopolancecofficial/DeepLearningProjects/blob/main/ComputerVision/SemanticSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from scipy import io
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import os

In [2]:
!git clone https://github.com/bearpaw/clothing-co-parsing.git

Cloning into 'clothing-co-parsing'...
remote: Enumerating objects: 4234, done.
remote: Total 4234 (delta 0), reused 0 (delta 0), pack-reused 4234
Receiving objects: 100% (4234/4234), 124.59 MiB | 47.04 MiB/s, done.
Resolving deltas: 100% (1100/1100), done.


In [5]:
!ls clothing-co-parsing/

annotations  label_list.mat  photos	show_image_anno.m
example.jpg  LICENSE	     README.md	show_pixel_anno.m


In [6]:
img_path = "./clothing-co-parsing/photos/"
mask_path = "./clothing-co-parsing/annotations/pixel-level/"

In [94]:
def read_images(path):

  images = []
  for img_path in os.listdir(path):
    url = os.path.join(path, img_path)
    img = cv2.imread(url)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(tf.convert_to_tensor(img))
  
  return images

In [95]:
images = read_images(img_path)

In [17]:
"""
image_content = tf.io.read_file(image_paths)
mask_content = tf.io.read_file(mask_paths)

images = tf.image.decode_jpeg(image_content, channels=self.channels[0])
masks = tf.image.decode_jpeg(mask_content, channels=self.channels[1])
"""

@tf.function
def load_image(filename):
  raw = tf.io.read_file(filename)
  image = tf.image.decode_png(raw, channels=3) #for mask channels = 1, decode_jpeg for jpg images
  # the `print` executes during tracing.
  print("Initial shape: ", image.shape)
  image.set_shape([28, 28, 3])
  print("Final shape: ", image.shape)
  return image

'\nimage_content = tf.io.read_file(image_paths)\nmask_content = tf.io.read_file(mask_paths)\n\nimages = tf.image.decode_jpeg(image_content, channels=self.channels[0])\nmasks = tf.image.decode_jpeg(mask_content, channels=self.channels[1])\n'

In [15]:
!pip install tensorflow_addons

     |████████████████████████████████| 686kB 27.3MB/s 


In [16]:
import tensorflow_addons as tfa

In [20]:
def blur(image, label):
  image = tfa.image.gaussian_filter2d(image=image, filter_shape=(11, 11), sigma=0.8)
  return image, label

In [47]:
raw = tf.io.read_file("./001.png")
mask = tf.image.decode_png(raw, channels=1)

In [57]:
mask = tf.reshape(mask, [mask.shape[0], mask.shape[1]])
mask

<tf.Tensor: shape=(4000, 6000), dtype=uint8, numpy=
array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0, 70, ...,  0,  0,  0],
       [ 0,  0, 70, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)>

In [66]:
np.argmax(mask, axis=1)

array([3727, 3619,    4, ..., 1832, 1831,    0])

In [41]:
label = tf.one_hot(mask, 3)

In [42]:
label

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [68]:
def load_masks(path, num_of_classes):

  masks = []
  for mask_path in os.listdir(path):
    url = os.path.join(path, mask_path)
    raw = tf.io.read_file(url)
    mask = tf.image.decode_png(raw, channels=1)
    mask = tf.reshape(mask, [mask.shape[0], mask.shape[1]])
    mask = tf.one_hot(mask, num_of_classes)
    masks.append(mask)
  
  return masks

In [101]:
def load_masks_mat(path, num_of_classes):

  masks = []
  for mask_path in os.listdir(path):
    url = os.path.join(path, mask_path)
    mask_file = io.loadmat(url)
    mask = tf.convert_to_tensor(mask_file['groundtruth'])
    mask = tf.one_hot(mask, num_of_classes)
    masks.append(mask)
  
  return masks

In [102]:
masks = load_masks_mat(mask_path, 4)

In [103]:
print(masks[0].shape)
print(images[0].shape)

(827, 550, 4)
(819, 550, 3)


In [104]:
def resize_image(image):
     image = tf.cast(image, tf.float32)
     # scale values to [0,1]
     image = image/255.0
     # resize image
     image = tf.image.resize(image, (128,128))
     return image 

In [105]:
def resize_mask(mask):
     mask = tf.image.resize(mask, (128,128))
     mask = tf.cast(mask, tf.uint8)
     return mask 

In [106]:
x = [resize_image(i) for i in images]
y = [resize_mask(m) for m in masks]

In [107]:
print(x[0].shape)
print(y[0].shape)

(128, 128, 3)
(128, 128, 4)


In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X,train_y, val_y = train_test_split(X,y, 
                                                 test_size=0.2, 
                                                 random_state=0
                                                )
train_X = tf.data.Dataset.from_tensor_slices(train_X)
val_X = tf.data.Dataset.from_tensor_slices(val_X)
train_y = tf.data.Dataset.from_tensor_slices(train_y)
val_y = tf.data.Dataset.from_tensor_slices(val_y)
train_X.element_spec, train_y.element_spec, val_X.element_spec, val_y.element_spec

In [ ]:
train = tf.data.Dataset.zip((train_X, train_y))
val = tf.data.Dataset.zip((val_X, val_y)) 

In [ ]:
def brightness(img, mask):
     img = tf.image.adjust_brightness(img, 0.1)
     return img, mask
 
 def gamma(img, mask):
     img = tf.image.adjust_gamma(img, 0.1)
     return img, mask

 def hue(img, mask):
     img = tf.image.adjust_hue(img, -0.1)
     return img, mask

 def crop(img, mask):
     img = tf.image.central_crop(img, 0.7)
     img = tf.image.resize(img, (128,128))
     mask = tf.image.central_crop(mask, 0.7)
     mask = tf.image.resize(mask, (128,128))
     mask = tf.cast(mask, tf.uint8)
     return img, mask

 def flip_hori(img, mask):
     img = tf.image.flip_left_right(img)
     mask = tf.image.flip_left_right(mask)
     return img, mask

 def flip_vert(img, mask):
     img = tf.image.flip_up_down(img)
     mask = tf.image.flip_up_down(mask)
     return img, mask

 def rotate(img, mask):
     img = tf.image.rot90(img)
     mask = tf.image.rot90(mask)
     return img, mask

In [ ]:
# perform augmentation on train data only
 a = train.map(brightness)
 b = train.map(gamma)
 c = train.map(hue)
 d = train.map(crop)
 e = train.map(flip_hori)
 f = train.map(flip_vert)
 g = train.map(rotate)

 train = train.concatenate(a)
 train = train.concatenate(b)
 train = train.concatenate(c)
 train = train.concatenate(d)
 train = train.concatenate(e)
 train = train.concatenate(f)
 train = train.concatenate(g)

In [ ]:
BATCH = 64
AT = tf.data.AUTOTUNE
BUFFER = 1000
STEPS_PER_EPOCH = 800//BATCH
VALIDATION_STEPS = 200//BATCH
train = train.cache().shuffle(BUFFER).batch(BATCH).repeat()
train = train.prefetch(buffer_size=AT)
val = val.batch(BATCH) 